In [ ]:
import climepi  # noqa
import holoviews as hv
import numpy as np
from climepi import climdata, epimod

import opts
from figure_export import export_sensitivity_figure

# Get options
options = opts.get_opts()
year_range = options["year_range"]
location = options["location_default"]
epi_model_name = options["epi_model_name_default"]
uncertainty_level = options["uncertainty_level"]
polyfit_degree = options["polyfit_degree"]
plot_opts_clim = options["plot_opts_clim"]
plot_opts_epi = options["plot_opts_epi"]
data_base_dir = options["data_base_dir"]

In [2]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset("isimip_cities_daily", base_dir=data_base_dir)
    .sel(location=location)
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
)
# Yearly average temperature data
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")
ds_temp_yearly["time"] = ds_temp_yearly["time.year"]  # avoids plotting issues

In [ ]:
epi_model_base = epimod.get_example_model(epi_model_name)
temperature_range_base = epi_model_base.temperature_range
temperature_range_narrow = (
    temperature_range_base[0] + 2,
    temperature_range_base[1] - 2,
)
temperature_range_wide = (
    temperature_range_base[0] - 2,
    temperature_range_base[1] + 2,
)
epi_model_narrow = epimod.SuitabilityModel(temperature_range=temperature_range_narrow)
epi_model_wide = epimod.SuitabilityModel(temperature_range=temperature_range_wide)
print(
    f"Base suitability range: {temperature_range_base[0]}-{temperature_range_base[1]}°C"
)
print(
    "Narrower suitability range: "
    f"{temperature_range_narrow[0]}-{temperature_range_narrow[1]}°C"
)
print(
    f"Wider suitability range: "
    f"{temperature_range_wide[0]}-{temperature_range_wide[1]}°C"
)

In [4]:
# Epi variability - plume
panels = []
for epi_model, show_legend in zip([epi_model_narrow, epi_model_wide], [True, False]):
    ds_suitability = ds_clim.climepi.run_epi_model(epi_model)
    ds_days_suitable = ds_suitability.climepi.yearly_portion_suitable()
    ds_days_suitable["time"] = ds_days_suitable["time.year"]
    panels += [
        ds_days_suitable.climepi.plot_uncertainty_interval_decomposition(
            uncertainty_level=uncertainty_level, polyfit_degree=polyfit_degree
        ).opts(
            **{
                **plot_opts_epi,
                "ylim": (0, 240),
                "yticks": np.arange(241, step=30),
                "show_legend": show_legend,
            }
        )
    ]

In [ ]:
# Show panels
hv.Layout(panels).cols(1)

In [6]:
# Combine panels
export_sensitivity_figure(panels)